In [7]:
pip install mistralai

Our API is currently available through [La Plateforme](https://console.mistral.ai/). You need to activate payments on your account to enable your API keys. After a few moments, you will be able to use our `chat` endpoint:

Or our embeddings endpoint:

In [ ]:
from mistralai import Mistral

api_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
model = "mistral-large-latest"

client = Mistral(api_key=api_key)

In [ ]:
print(model)

mistral-large-latest


In [ ]:
from mistralai import Mistral

api_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
model = "mistral-small-latest"

client = Mistral(api_key=api_key)

In [ ]:
PROMPT_TEMPLATE = """
You are a professional travel email assistant.

TASK 1: EMAIL CORRECTION
- Rewrite the email with correct spelling, grammar, and formatting.
- Do NOT add new information.

TASK 2: STRICT INFORMATION EXTRACTION
- Extract ONLY what is explicitly mentioned in the email.
- Do NOT guess or infer missing information.
- If a field is missing, set it to null.
- Extract a single start date (if a range is mentioned, take only the start date).
- If multiple hotels are mentioned, return them as an array.
- If days or nights are explicitly mentioned in the email, extract them; else set them to null.
- If both start_date and end_date are present, calculate:
    - duration = (end_date - start_date) + 1
  Otherwise, set duration = null.
- Attractions:
  • Capture all attractions/activities explicitly mentioned.
  • INCLUDE only sightseeing places
  • EXCLUDE meals, shopping, transfers, checkout, packages, timings

OUTPUT JSON FORMAT:

{{
  "corrected_email": "",
  "extracted_info": {{
    "name": null,
    "pax": null,
    "city": null,
    "start_date": null,
    "end_date": null,
    "days": null,
    "nights": null,
    "duration": null,
    "hotel_type": null,
    "hotel_name": [],
    "attractions": []
  }}
}}

EMAIL:
{email_text}
"""

In [ ]:
def process_travel_email(email_text: str) -> str:
    """
    Takes an email text and returns Mistral's response containing
    corrected email and extracted information in JSON.
    """
    prompt = PROMPT_TEMPLATE.format(email_text=email_text)
    response = client.chat.complete(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    # Return the model's JSON string directly
    return response.choices[0].message.content

In [ ]:
# --- USAGE EXAMPLE ---
email_text = """
Kind attn Sir,
Greetings from arunesh Travel Services !!! Chennai,
Pls give rates for the below

Pax Name    :- Mr. G V Madhavakrishna x 2 pax- 1 DUBLE ROOM
check in       :- 03-07 Feb 26  04nights/05 days
Flight Time   :-MELBOURNE-SIN 03feb   SQ 228    1640/2120
                      SINGAPORE-MAA 07feb  TR 8414   2050/2240
Hotel            :- 04 star Hotel (boss Hotel/Hilton garden/village bugis hotel,Littel                          India AREA)

DAY 01- ARRVL TRANSFER TO HOTEL
DAY 02- CITY TOUR OF SINGAPORE/ GARDEN BY THE BAY/SANTHOSA ISLAND                 5 IN-ONE PAKG
DAY 03- FULL DAY UNIVERSAL STUDIO WITH LUNCH
DAY 04 -JURONG BIRD PARK-MBS SKY PARK -- EVENING NIGHT SAFARI
DAY O5- FULL DAY SHOPPING-1200 HRS CHECK OUT-1630 DEP TO AIR PORT

PLS   REPLY WITH GOOD RATES
"""

result_json = process_travel_email(email_text)
print(result_json)

```json
{
  "corrected_email": "Kind Attention Sir,\n\nGreetings from Arunesh Travel Services, Chennai!\n\nPlease provide rates for the below:\n\nPax Name: Mr. G V Madhavakrishna x 2 pax - 1 Double Room\nCheck-in: 03 Feb 26, 04 nights / 05 days\nFlight Time:\n- MELBOURNE-SIN, 03 Feb, SQ 228, 1640/2120\n- SINGAPORE-MAA, 07 Feb, TR 8414, 2050/2240\nHotel: 4-star Hotel (Boss Hotel / Hilton Garden / Village Bugis Hotel, Little India area)\n\nDay 01: Arrival transfer to hotel\nDay 02: City tour of Singapore / Gardens by the Bay / Sentosa Island (5-in-one package)\nDay 03: Full day Universal Studios with lunch\nDay 04: Jurong Bird Park / Marina Bay Sands SkyPark / Evening Night Safari\nDay 05: Full day shopping, 1200 hrs check-out, 1630 departure to airport\n\nPlease reply with good rates.",
  "extracted_info": {
    "name": "Mr. G V Madhavakrishna",
    "pax": 2,
    "city": "Singapore",
    "start_date": "2026-02-03",
    "end_date": "2026-02-07",
    "days": 5,
    "nights": 4,
    "durat

```json
{
  "corrected_email": "Kind Attention Sir,\n\nGreetings from Arunesh Travel Services, Chennai!\n\nPlease provide rates for the below:\n\nPax Name: Mr. G V Madhavakrishna x 2 pax – 1 Double Room\nCheck-in: 03 Feb 2026 – 04 nights / 05 days\nFlight Time:\n- MELBOURNE-SIN 03 Feb – SQ 228 – 16:40 / 21:20\n- SINGAPORE-MAA 07 Feb – TR 8414 – 20:50 / 22:40\nHotel: 3-star hotel (Boss Hotel / Hilton Garden / Village Bugis Hotel, Little India area)\n\nDay 01: Arrival transfer to hotel\nDay 02: City tour of Singapore / Gardens by the Bay / Sentosa Island – 5-in-1 package\nDay 03: Full-day Universal Studios with lunch\nDay 04: Jurong Bird Park / MBS SkyPark / Evening Night Safari\nDay 05: Full-day shopping – 12:00 hrs check-out – 16:30 departure to airport\n\nPlease reply with good rates.",
  "extracted_info": {
    "name": "Mr. G V Madhavakrishna",
    "pax": 2,
    "city": "Singapore",
    "start_date": "2026-02-03",
    "end_date": "2026-02-07",
    "duration_days": 5,
    "duration_nights": 4,
    "hotel_type": "3-star",
    "hotel_name": ["Boss Hotel", "Hilton Garden", "Village Bugis Hotel"],
    "attractions": [
      "Arrival transfer to hotel",
      "City tour of Singapore",
      "Gardens by the Bay",
      "Sentosa Island",
      "Universal Studios",
      "Jurong Bird Park",
      "MBS SkyPark",
      "Night Safari",
      "Shopping"
    ]
  }
}
```
